In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import scipy

In [2]:
import matplotlib.pyplot as plt

# Import all provincial timeseries

In [3]:
import glob

not_timeseries = ['./Update/COVID-19/dati-province/dpc-covid19-ita-province.csv',
                  './Update/COVID-19/dati-province/dpc-covid19-ita-province-latest.csv']
glued_data = pd.DataFrame()
for file_name in glob.glob('./Update/COVID-19/dati-province/'+'*.csv'):
    if file_name not in not_timeseries:
        x = pd.read_csv(file_name, low_memory=False)
        glued_data = pd.concat([glued_data,x],axis=0)

# Read remaining:
- latest
- province

In [4]:
provinces_latest_pd = pd.read_csv('./Update/COVID-19/dati-province/dpc-covid19-ita-province-latest.csv', low_memory=False)
provinces_pd = pd.read_csv('./Update/COVID-19/dati-province/dpc-covid19-ita-province.csv', low_memory=False)

In [5]:
provinces_latest_pd

data stato  codice_regione denominazione_regione  \
0    2020-03-30T17:00:00   ITA              13               Abruzzo   
1    2020-03-30T17:00:00   ITA              13               Abruzzo   
2    2020-03-30T17:00:00   ITA              13               Abruzzo   
3    2020-03-30T17:00:00   ITA              13               Abruzzo   
4    2020-03-30T17:00:00   ITA              13               Abruzzo   
..                   ...   ...             ...                   ...   
123  2020-03-30T17:00:00   ITA               5                Veneto   
124  2020-03-30T17:00:00   ITA               5                Veneto   
125  2020-03-30T17:00:00   ITA               5                Veneto   
126  2020-03-30T17:00:00   ITA               5                Veneto   
127  2020-03-30T17:00:00   ITA               5                Veneto   

     codice_provincia               denominazione_provincia sigla_provincia  \
0                  69                                Chieti              CH   
1                  66                              L'Aquila              AQ   
2                  68                               Pescara              PE   
3                  67                                Teramo              TE   
4                 979  In fase di definizione/aggiornamento             NaN   
..                ...                                   ...             ...   
123                26                               Treviso              TV   
124                27                               Venezia              VE   
125                23                                Verona              VR   
126                24                               Vicenza              VI   
127               999  In fase di definizione/aggiornamento             NaN   

           lat       long  totale_casi  note_it  note_en  
0    42.351032  14.167546          273      NaN      NaN  
1    42.351222  13.398438          115      NaN      NaN  
2    42.464584  14.213648          594      NaN      NaN  
3    42.658918  13.704400          363      NaN      NaN  
4     0.000000   0.000000            0      NaN      NaN  
..         ...        ...          ...      ...      ...  
123  45.667546  12.245074         1449      NaN      NaN  
124  45.434905  12.338452         1107      NaN      NaN  
125  45.438390  10.993527         1982      NaN      NaN  
126  45.547497  11.545971         1168      NaN      NaN  
127   0.000000   0.000000          276      NaN      NaN  

[128 rows x 12 columns]

# Plots

In [6]:
regions = provinces_pd.denominazione_regione.unique()

In [7]:
provinces_pd['data']= pd.to_datetime(provinces_pd['data'])

In [8]:
provinces_pd.totale_casi.max()

8676

In [9]:
fig, ax = plt.subplots(nrows=7, ncols=3, figsize=(35,50))

i = 0
j = 0
for region in regions:
    pd_r = provinces_pd[provinces_pd.denominazione_regione == region]
    provinces = pd_r.denominazione_provincia.unique().tolist()
    #if "In fase di definizione/aggiornamento" in str(provinces):
    provinces.remove("In fase di definizione/aggiornamento")

    ax[i,j].set_title(region, fontsize=20)
    for province in provinces:
        df = pd_r[pd_r.denominazione_provincia == province].sort_values('data')
        ax[i,j].plot(df.data.dt.date, df.totale_casi, '-o', label=province)
        ax[i,j].tick_params('x', labelrotation=45)
        ax[i,j].tick_params(axis='both', which='major', labelsize=16)
        ax[i,j].tick_params(axis='both', which='minor', labelsize=16)
        ax[i,j].set_ylim(0,provinces_pd.totale_casi.max())
        #ax[i,j].set_yscale('log')
        ax[i,j].legend(fontsize=16, loc='upper left')
        ax[i,j].grid(True)
        ax[i,j].set_ylabel('Total number of cases', fontsize=16)
    if j == 2:
        j = 0
        i += 1
    else:
        j += 1
fig.tight_layout()

# Read regional data

In [10]:
## Select regional data ##
regions_latest_pd = pd.read_csv('./Update/COVID-19/dati-regioni/dpc-covid19-ita-regioni-latest.csv', low_memory=False)

In [11]:
## Make the groupBy at regional level from the province dataframe ##
provinces_grouped = provinces_latest_pd.groupby(['data', 'denominazione_regione']).agg({'totale_casi':'sum'})\
                                       .reset_index()
provinces_grouped.rename(columns={'totale_casi': 'totale_casi_pr'}, inplace=True)

provinces_grouped['data']= pd.to_datetime(provinces_grouped['data'])

In [12]:
regions_latest_pd_selection = regions_latest_pd[['data', 'denominazione_regione',
       'totale_attualmente_positivi', 'nuovi_attualmente_positivi',
       'dimessi_guariti', 'deceduti', 'totale_casi', 'tamponi']]
regions_latest_pd_selection['data']= pd.to_datetime(regions_latest_pd_selection['data'])

/Users/davide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
regions_latest_pd_selection

data  denominazione_regione  totale_attualmente_positivi  \
0  2020-03-30 17:00:00                Abruzzo                         1169   
1  2020-03-30 17:00:00             Basilicata                          208   
2  2020-03-30 17:00:00           P.A. Bolzano                         1098   
3  2020-03-30 17:00:00               Calabria                          602   
4  2020-03-30 17:00:00               Campania                         1739   
5  2020-03-30 17:00:00         Emilia Romagna                        10766   
6  2020-03-30 17:00:00  Friuli Venezia Giulia                         1109   
7  2020-03-30 17:00:00                  Lazio                         2497   
8  2020-03-30 17:00:00                Liguria                         2383   
9  2020-03-30 17:00:00              Lombardia                        25006   
10 2020-03-30 17:00:00                 Marche                         3251   
11 2020-03-30 17:00:00                 Molise                          107   
12 2020-03-30 17:00:00               Piemonte                         7655   
13 2020-03-30 17:00:00                 Puglia                         1585   
14 2020-03-30 17:00:00               Sardegna                          622   
15 2020-03-30 17:00:00                Sicilia                         1408   
16 2020-03-30 17:00:00                Toscana                         4050   
17 2020-03-30 17:00:00            P.A. Trento                         1357   
18 2020-03-30 17:00:00                 Umbria                          834   
19 2020-03-30 17:00:00          Valle d'Aosta                          518   
20 2020-03-30 17:00:00                 Veneto                         7564   

    nuovi_attualmente_positivi  dimessi_guariti  deceduti  totale_casi  \
0                            0               74       102         1345   
1                           11                1         5          214   
2                           64              153        74         1325   
3                           25               14        31          647   
4                          183               88       125         1952   
5                          231             1227      1538        13531   
6                          -32              285       107         1501   
7                          135              267       150         2914   
8                          104              437       397         3217   
9                         -386            10337      6818        42161   
10                          91               16       417         3684   
11                           7               18         9          134   
12                         387              308       749         8712   
13                         153               36        91         1712   
14                          40               32        28          682   
15                          78               71        76         1555   
16                         264              131       231         4412   
17                          64              178       147         1682   
18                         -63              184        33         1051   
19                         -21               16        50          584   
20                         313              747       413         8724   

    tamponi  
0      7948  
1      1833  
2     10640  
3      9013  
4     12969  
5     50990  
6     14003  
7     32846  
8      9677  
9    111057  
10    10979  
11      955  
12    25478  
13    13117  
14     4993  
15    14758  
16    30099  
17     6377  
18     8150  
19     1536  
20    99941

In [15]:
bad = provinces_latest_pd[provinces_latest_pd.denominazione_provincia == 'In fase di definizione/aggiornamento']\
                        .sort_values(['data','denominazione_regione'])
bad_df = bad[['data','denominazione_regione']].sort_values(['data','denominazione_regione'])
bad_df['In fase di definizione/aggiornamento']  = bad['totale_casi']

In [16]:
final = regions_latest_pd_selection.merge(provinces_grouped, on=['data', 'denominazione_regione'])
#final = final.merge(bad_df, on=['data', 'denominazione_regione'])

In [17]:
final_1 = final.merge(bad_df, on=['denominazione_regione'])

In [18]:
final_1['percentage_aggiornamento'] = 100*final_1['In fase di definizione/aggiornamento']/final_1.totale_casi_pr

In [19]:
final_1.drop('data_y', axis=1, inplace=True)

In [20]:
final_1.rename(columns={'data_x':'data', 'totale_casi_pr':'totale_casi_calcolati_sommando_su_provincia'}, inplace=True)

In [21]:
final_1

data  denominazione_regione  totale_attualmente_positivi  \
0  2020-03-30 17:00:00                Abruzzo                         1169   
1  2020-03-30 17:00:00             Basilicata                          208   
2  2020-03-30 17:00:00           P.A. Bolzano                         1098   
3  2020-03-30 17:00:00               Calabria                          602   
4  2020-03-30 17:00:00               Campania                         1739   
5  2020-03-30 17:00:00         Emilia Romagna                        10766   
6  2020-03-30 17:00:00  Friuli Venezia Giulia                         1109   
7  2020-03-30 17:00:00                  Lazio                         2497   
8  2020-03-30 17:00:00                Liguria                         2383   
9  2020-03-30 17:00:00              Lombardia                        25006   
10 2020-03-30 17:00:00                 Marche                         3251   
11 2020-03-30 17:00:00                 Molise                          107   
12 2020-03-30 17:00:00               Piemonte                         7655   
13 2020-03-30 17:00:00                 Puglia                         1585   
14 2020-03-30 17:00:00               Sardegna                          622   
15 2020-03-30 17:00:00                Sicilia                         1408   
16 2020-03-30 17:00:00                Toscana                         4050   
17 2020-03-30 17:00:00            P.A. Trento                         1357   
18 2020-03-30 17:00:00                 Umbria                          834   
19 2020-03-30 17:00:00          Valle d'Aosta                          518   
20 2020-03-30 17:00:00                 Veneto                         7564   

    nuovi_attualmente_positivi  dimessi_guariti  deceduti  totale_casi  \
0                            0               74       102         1345   
1                           11                1         5          214   
2                           64              153        74         1325   
3                           25               14        31          647   
4                          183               88       125         1952   
5                          231             1227      1538        13531   
6                          -32              285       107         1501   
7                          135              267       150         2914   
8                          104              437       397         3217   
9                         -386            10337      6818        42161   
10                          91               16       417         3684   
11                           7               18         9          134   
12                         387              308       749         8712   
13                         153               36        91         1712   
14                          40               32        28          682   
15                          78               71        76         1555   
16                         264              131       231         4412   
17                          64              178       147         1682   
18                         -63              184        33         1051   
19                         -21               16        50          584   
20                         313              747       413         8724   

    tamponi  totale_casi_calcolati_sommando_su_provincia  \
0      7948                                         1345   
1      1833                                          214   
2     10640                                         1325   
3      9013                                          647   
4     12969                                         1952   
5     50990                                        13531   
6     14003                                         1501   
7     32846                                         2914   
8      9677                                         3217   
9    111057                                        42161   
10    10979 

In [98]:
final_1.to_csv('Covid_comparison_30Mar2019.csv')